In [1]:
import os

In [2]:
%pwd

'c:\\Users\\saad_\\onedrive\\bureau\\truckClassification\\TruckClassification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\saad_\\onedrive\\bureau\\truckClassification\\TruckClassification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'cnnClassifier'